# 1. Set Environments

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="Machin Translator_01"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

In [2]:
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 2. Set Datasets

In [3]:
from datasets import Dataset, DatasetDict
import pickle
import os
import time


with open('./data/train_data_300.pkl', 'rb') as file:
    train_data = pickle.load(file)
len(train_data)

with open('./data/validation_data_28.pkl', 'rb') as file:
    test_data = pickle.load(file)
len(test_data)

train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

# DatasetDict로 "train"과 "test" 데이터셋 묶기
dataset_dict = DatasetDict({
        'train': train_dataset,
        'test': test_dataset
    })

# 3. Checkpoints Listing

In [4]:
checkpoint_dir = "./checkpoint/translate_machine_llama3ko_intsuct_origindata300_02"

# 디렉터리 이름에서 숫자를 추출하는 함수
def extract_number(directory_name):
    return int(directory_name.split('-')[-1])


# 디렉터리 항목을 숫자 순서대로 정렬
checkpoints = []
with os.scandir(checkpoint_dir) as entries:
    for entry in entries:
        if entry.is_dir():
            checkpoints.append(entry.name)

# 숫자 순서대로 정렬
checkpoints.sort(key=extract_number)

# 4. Evaluate

In [5]:
import evaluate
import re
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

metric = evaluate.load("sacrebleu")

model_id = "beomi/Llama-3-KoEn-8B-Instruct-preview"
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"

def make_prompt(text):
    return f'''Translate input sentence to Korean
### Input: {text}
### Translated:'''


def clean_text(text):
    text = re.sub(r'\(.*?\)', '', text)
    return text.strip()


def counter_terms(terms, text):
    if not isinstance(terms, list):
        terms = terms.split(", ")

    return sum(text.lower().count(term.lower()) for term in terms)


def generate_and_evaluate(data, model, tokenizer):
    prompt = make_prompt(data['english'])
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=1024, pad_token_id=tokenizer.pad_token_id)
    prediction = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)

    label_clean = clean_text(data['korean'])
    prediction_clean = clean_text(prediction)

    input_count = counter_terms(data['terms'], data['english'])
    predic_count = counter_terms(data['terms'], prediction)

    weight = 1.0 if predic_count > input_count else predic_count / input_count

    result = metric.compute(predictions=[prediction_clean], references=[label_clean])

    return result['score'], weight, weight * result['score']

# 시작 시간 기록
start_time = time.time()

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map='auto', cache_dir=cache_model_dir)

bleu_scores = {}

for checkpoint in checkpoints[10:]:
    print('#' * 50)
    print(checkpoint)
    print('#' * 50)

    peft_model_id = os.path.join(checkpoint_dir, checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
    model.resize_token_embeddings(len(tokenizer))

    loaded_model = PeftModel.from_pretrained(model=model, model_id=peft_model_id)

    results = [generate_and_evaluate(data, loaded_model, tokenizer) for data in dataset_dict['test']]

    bleus, weights, weighted_bleus = zip(*results)

    score = {
        "bleu": np.mean(bleus),
        "weight": np.mean(weights),
        "weighted_bleus": np.mean(weighted_bleus)
    }

    bleu_scores[checkpoint] = score

    del peft_model_id, tokenizer, loaded_model

df = pd.DataFrame.from_dict(bleu_scores, orient='index')

# 종료 시간 기록
end_time = time.time()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

##################################################
checkpoint-110
##################################################


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##################################################
checkpoint-120
##################################################


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##################################################
checkpoint-290
##################################################


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##################################################
checkpoint-300
##################################################


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
print(df)

# 실행 시간 계산
execution_time = end_time - start_time

print(f"실행 시간: {execution_time} 초")

                     bleu    weight  weighted_bleus
checkpoint-110  47.655708  0.783929       38.020108
checkpoint-120  48.512326  0.868778       42.207067
checkpoint-130  47.967690  0.883362       42.108866
checkpoint-140  49.140856  0.860785       42.241602
checkpoint-150  48.969970  0.921641       44.913353
checkpoint-160  48.083907  0.888605       42.533465
checkpoint-170  48.533218  0.855456       41.175765
checkpoint-180  48.648087  0.822208       39.730117
checkpoint-190  50.512783  0.836097       42.451408
checkpoint-200  48.490799  0.896896       43.562024
checkpoint-210  48.102794  0.882015       43.000055
checkpoint-220  49.040487  0.850425       41.481763
checkpoint-230  48.273156  0.900085       43.524116
checkpoint-240  51.532903  0.858078       44.728627
checkpoint-250  51.716001  0.852126       44.119474
checkpoint-260  50.759023  0.840363       42.638950
checkpoint-270  49.657846  0.904549       44.878553
checkpoint-280  49.732695  0.891156       44.511654
checkpoint-2